In [ ]:
# import tensorflow as tf
# from tensorflow.keras import backend as K
# from tensorflow.keras.losses import mean_squared_error

# def approx_loss(y_pred, params):
#     y_lower = tf.constant(params['y_l'], shape=(len(y_pred), 1))
#     y_upper = tf.constant(params['y_u'], shape=(len(y_pred), 1))
    
#     return K.sum(K.relu(y_lower - y_pred) + K.relu(y_pred - y_upper))

                 
# def combined_loss(params):
#     # wrapper function needed for the custom loss function to be accepted from keras
#     def loss(y_true, y_pred):
#         data_based_loss = mean_squared_error(y_true, y_pred)
#         approximation_loss = approx_loss(y_pred, params)
#         lambda_A = params['lambda_A']

#         return data_based_loss + lambda_A * approximation_loss
#     return fun

# HYPER_PARAMS = {'y_l':3,
#                 'y_u':4,
#                 'lambda_A': 0.5,
#                 }

# approx_loss([0, 1, 2, 3], HYPER_PARAMS)

In [ ]:
import numpy as np
import tensorflow as tf

import data_preprocessing as util
import lstm_model as lstm

tf.compat.v1.set_random_seed(1)

### Set Hyperparameters

In [ ]:
HYPER_PARAMS = {'n_epochs': 10,
                'd_sample': 1,
                'gauss_sigma': 10 , 
                'n_steps': 50,
                'n_features':2,
                'n_lstm_units_1': 50,
                'alpha_1': 0.1,
                'n_lstm_units_2': 20,
                'alpha_2': 0.1,
                'n_dense_units': 10,
                'activation': 'tanh',
                'dropout': 0.2,
                'learning_rate': 0.001,
                'early_stopping': 20,
                'optimizer': 'Adam',
                'loss': 'mse',
                'metric': 'mae',
                'y_l':3,
                'y_u':4,
                'lambda_A': 0.5,
                }

### Prepare Training Data

In [ ]:
X_train, y_train, scalers_train = util.prepare_data(HYPER_PARAMS, ['Profile 10A'], 0, 4)

### Initialize and Train Model

In [ ]:
lstm = lstm.Model()
lstm.initialize(HYPER_PARAMS)

In [ ]:
lstm.train(X_train, y_train, scalers_train)

# save model and hyperparameters
MODEL_ID = str(np.random.randint(10000))
lstm.model.save('trained_models/' + str(MODEL_ID))
np.save('trained_models/' + str(MODEL_ID) + '/hyperparameters', HYPER_PARAMS)

### Prepare Validation & Test Data

In [ ]:
X_validation, y_validation, scalers_validation = util.prepare_data(HYPER_PARAMS, ['Profile 10A'], 0, 4)
X_test, y_test, scalers_test = util.prepare_data(HYPER_PARAMS, ['Profile 10A'], 0, 4)

scalers = scalers_train, scalers_validation, scalers_test 
# scalers shape: ((train_cur, train_volt), (val_cur, val_volt), (test_cur, test_volt))

### Test Model

In [ ]:
lstm.test(X_train, y_train, X_validation, y_validation, X_test, y_test, scalers)